<a href="https://colab.research.google.com/github/ArtHommage/HommageTools/blob/main/notebooks/comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [1]:
# Cell 1: Upgrade Core Tools (Run this cell first)
# -------------------------------------------------
print("Upgrading pip...")
!pip install --upgrade pip

print("\nUpgrading setuptools...")
!pip install --upgrade setuptools

print("\n---------------------------------------------------------------------")
print("IMPORTANT: Core tools upgraded.")
print("Please RESTART THE RUNTIME now using the menu options before proceeding.")
print("(e.g., Runtime -> Restart runtime in Google Colab)")
print("---------------------------------------------------------------------")

Upgrading pip...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 68.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2

Upgrading setuptools...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.



---------------------------------------------------------------------
IMPORTANT: Core tools upgraded.
Please RESTART THE RUNTIME now using the menu options before proceeding.
(e.g., Runtime -> Restart runtime in Google Colab)
---------------------------------------------------------------------


In [1]:
# Cell 2: Install Dependencies (Run this cell AFTER restarting the runtime)
# -----------------------------------------------------------------------
print("Installing jedi (for ipython)...")
!pip install "jedi>=0.16"

print("\nInstalling specific NumPy version required by dependencies...")
# This version range should satisfy thinc, tensorflow, numba, ultralytics etc.
!pip install "numpy>=2.0.0,<2.1.0"

print("\nInstalling main project packages...")
# List all your main packages here. Combining them helps pip resolve dependencies.
# Adding piexif and albumentations based on your previous install list.
!pip install \
    transparent_background \
    piexif \
    ultralytics \
    surrealist \
    blend_modes \
    gguf \
    dynamicprompts \
    pygtrans \
    segment_anything \
    dill \
    boto3 \
    fake_useragent \
    redis \
    fal_client \
    replicate \
    psd-tools \
    onnxruntime \
    oidn \
    color-matcher \
    albumentations \
    omegaconf \
    pytorch_lightning \
    open_clip_torch \
    xformers \
    transformers==4.49.0

print("\n---------------------------------------------------------------------")
print("Dependencies installation attempted.")
print("Check output above for any errors or new dependency conflicts.")
print("---------------------------------------------------------------------")

Installing jedi (for ipython)...

Installing specific NumPy version required by dependencies...

Installing main project packages...

---------------------------------------------------------------------
Dependencies installation attempted.
Check output above for any errors or new dependency conflicts.
---------------------------------------------------------------------


In [ ]:
#!pip install transparent_background piexif ultralytics surrealist blend_modes gguf dynamicprompts pygtrans segment_anything dill boto3 fake_useragent redis fal_client replicate psd-tools onnxruntime oidn color-matcher albumentations --upgrade
#!pip install omegaconf pytorch_lightning open_clip_torch
#!pip uninstall xformers -y
#!pip install xformers
#!pip install --upgrade transformers==4.49.0
#!pip install "numpy>=2.0.0,<2.1.0" # Experimental

In [ ]:
#@title Environment Setup


OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI from main branch =-
  !git fetch origin
  !git checkout master
  !git reset --hard origin/master
  !git pull origin master

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

Mounting Google Drive...
/


Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server